In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ipyfilechooser import FileChooser
from joblib import dump, load


%matplotlib inline

In [ ]:
fc = FileChooser()
display(fc)


In [ ]:
data = pd.read_csv(os.path.join('..', 'simulator', 'logs', fc.selected))
data.describe()

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
label_movement = data.columns[0]

data[label_movement].value_counts().plot(kind='bar')
plt.title('actions')
plt.xlabel('stop')
plt.ylabel('Count')

In [ ]:
data_random_forward = data[data[label_movement] == 'forward'].sample(n=int(data.shape[0]/3))
data.drop(data_random_forward.index,inplace=True)
data[label_movement].value_counts().plot(kind='bar')
plt.title('actions')
plt.xlabel('action')
plt.ylabel('Count')


In [ ]:
train1 = data.drop([label_movement],axis=1)

In [ ]:
train1.head()

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf = SVC()

In [ ]:
data = data.dropna()

data_no_stop = data[data[label_movement] != 'stop']
conv_labels = [0 if values == 'forward' else values for values in data_no_stop[label_movement] ]
conv_labels = [1 if values == 'left' else values for values in conv_labels ]
conv_labels = [2 if values == 'right' else values for values in conv_labels ]

labels = conv_labels
train1 = data_no_stop.drop([ label_movement],axis=1)
data_no_stop.describe()


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train1, conv_labels, test_size = 0.10, random_state = 2)

In [ ]:
clf.fit(x_train,y_train)

In [ ]:
clf.score(x_test, y_test)


In [ ]:
dump(clf,os.path.join('..', 'models', 'auto_pilot.joblib'))


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

model = make_pipeline(Normalizer(), SVC())
model.fit(x_train,y_train)
model.score(x_test, y_test)

In [ ]:
plt.scatter(y_test, clf.predict(x_test))

In [ ]:
from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier(n_estimators = 100, learning_rate = 0.1)

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
clf.score(x_test, y_test)

In [ ]:
plt.scatter(y_test, clf.predict(x_test))

In [ ]:
dump(clf,os.path.join('..', 'models', 'auto_pilot.joblib'))


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble

parameters = {
    "learning_rate": [0.1, 0.01],
    "n_estimators": [100, 200]
}

clf = GridSearchCV(ensemble.GradientBoostingClassifier(), parameters, cv=3, n_jobs=-1)

clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))
print(clf.best_params_)

In [ ]:
dump(clf,os.path.join('..', 'models', 'auto_pilot.joblib'))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    "learning_rate": np.linspace(0.1, 0.001, 10),
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth": [3,5,8],
    "max_features": ["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample": [0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators": [10, 20, 50, 100, 200]
}

clf = RandomizedSearchCV(ensemble.GradientBoostingClassifier(), parameters, cv=3, n_jobs=-1, n_iter=10)

clf.fit(x_train, y_train)
print(clf.score(x_test, y_test))
print(clf.best_params_)

In [ ]:
dump(clf,os.path.join('..', 'models', 'auto_pilot.joblib'))
